# How to Use Weaviate Vector Database

1. Create new collection
2. Add new data (vector)
3. Retrieve data (hybrid)

## 1. Create new collection

In [2]:
import weaviate
import weaviate.classes.config as wc
from pprint import pprint


CONNECTION_CONFIG = {"port": 8080, "grpc_port": 50051, "skip_init_checks": True}


def add_collection(name, connection_config=CONNECTION_CONFIG):
    name = name.capitalize()

    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)

        collection = client.collections.create(
            name=name,
            vector_config=[
                wc.Configure.Vectors.self_provided(
                    name="custom_vector",
                    vector_index_config=wc.Configure.VectorIndex.hnsw(
                        ef_construction=300,
                        distance_metric=wc.VectorDistances.COSINE,
                    ),
                )
            ],
            properties=[wc.Property(name="text", data_type=wc.DataType.TEXT)],
            inverted_index_config=wc.Configure.inverted_index(  # Optional
                bm25_b=0.7,
                bm25_k1=1.25,
                index_null_state=True,
                index_property_length=True,
                index_timestamps=True,
            ),
        )

        print("New Collection created!")
        print(f" name: {collection.config.get().name}")

    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()


def delete_all_collections(connection_config=CONNECTION_CONFIG):
    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)
        client.collections.delete_all()
        print("All collections has been removed!")
    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()


def delete_collection(collections, connection_config=CONNECTION_CONFIG):
    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)
        client.collections.delete(collections)
        print(f"deleted collections: {collections}")
    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()


def list_collections(connection_config=CONNECTION_CONFIG):
    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)
        collections = client.collections.list_all(simple=True)
        return list(collections.keys())
    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()

/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update the gencode to avoid compatibility violations in the next ru

In [3]:
delete_all_collections()

All collections has been removed!


In [4]:
add_collection('document')

New Collection created!
 name: Document


In [5]:
list_collections()

['Document']

## 2. Add new data (vector)

In [6]:
from sentence_transformers import SentenceTransformer

# Load a multilingual embedding model from HuggingFace Model Hub
model = SentenceTransformer("intfloat/multilingual-e5-base")

/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def add_text(text, collection_name, connection_config=CONNECTION_CONFIG):
    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)
        documents = client.collections.get(collection_name)
        documents.data.insert(properties={"text": text}, vector=model.encode(text))
        print('New data is inserted!')
        print(f'text: {text}')
    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()

In [8]:
add_text('hello world', 'Document')

New data is inserted!
text: hello world


/Users/krimssmirk/Desktop/rag/.venv/lib/python3.11/site-packages/torch/nn/modules/module.py:1520: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [9]:
def list_objects(collection_name, connection_config=CONNECTION_CONFIG):
    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)
        documents = client.collections.get(collection_name)
        for item in documents.iterator(
            include_vector=True
        ):
            pprint(item)
    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()

In [10]:
list_objects('Document')

Object(uuid=_WeaviateUUIDInt('0855701d-e1e6-4c25-b56a-a91ef8757534'),
       metadata=MetadataReturn(creation_time=None,
                               last_update_time=None,
                               distance=None,
                               certainty=None,
                               score=None,
                               explain_score=None,
                               is_consistent=None,
                               rerank_score=None),
       properties={'text': 'hello world'},
       references=None,
       vector={'default': [0.019661959260702133,
                           0.03473087400197983,
                           -0.0027683284133672714,
                           0.020048990845680237,
                           0.024604743346571922,
                           -0.026798121631145477,
                           -0.019066978245973587,
                           -0.01449610386043787,
                           0.043844807893037796,
                         

## 2. Retrieve data (hybrid)

**Hybrid Search** = Keyword Search + Semantic Search
- An alpha of 1 is a pure vector search.
- An alpha of 0 is a pure keyword search.

In [11]:
def retrieve_documents(query, collection_name, connection_config=CONNECTION_CONFIG):
    client = None
    try:
        client = weaviate.connect_to_local(**connection_config)
        documents = client.collections.get(collection_name)
        response = documents.query.hybrid(
            query=query,
            alpha=0.6,  # Balance between keyword and vector
            target_vector="custom_vector",  # Specify which named vector to use for the vector component
            query_properties=[
                "text"
            ],  # Which properties to use for the keyword (BM25) search
            # IMPORTANT: If your collection has NO vectorizer_config at the class level
            # and only self_provided named vectors, you might need to also provide `vector`
            # within the hybrid query to explicitly give the query vector.
            # The Weaviate docs indicate `query` is used for both, but if your setup is strict
            # with `vectorizer_config=none()`, `query` won't be auto-vectorized.
            # In such cases, if you want semantic search as part of hybrid, you must provide the vector.
            vector=model.encode(query),  # This would be the vector of 'search_query'
            limit=5,
        )
        for object in response.objects:
            print(object.properties)
        
        if len(response.objects) == 0:
            print('there is no similar documents')
    except Exception as e:
        print(e)
    finally:
        if client:
            client.close()

In [16]:
query = '世界'

In [18]:
retrieve_documents(query, 'Document')

there is no similar documents
